In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Reading Data

In [5]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv")

# train_df = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv", nrows = 500000) # to read only half of the data

### Reducing Memory Usage

In [6]:

# this funreduce_mem_usagen will help to reduce momory 
# data will be smaller with the same value

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [7]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

### Summary of Data

In [10]:
# Summary = pd.DataFrame(train_df.dtypes, columns=['Dtype'])
# Summary["max"] = train_df.max()
# Summary["min"] = train_df.min()
# Summary["Null"] = train_df.isnull().sum() # to get null values
# Summary["First"] = train_df.iloc[0] # to get first value
# Summary["Second"] = train_df.iloc[1] # to get second value
# # Summary

### Fetaure Engineering

In [11]:
features = [col for col in train_df.columns if col not in ["target", "id"]]

In [12]:
train_df["min"] = train_df[features].min(axis = 1)
train_df["max"] = train_df[features].max(axis = 1)
train_df["mean"] = train_df[features].mean(axis = 1)
train_df["std"] = train_df[features].std(axis = 1)

test_df["min"] = test_df[features].min(axis = 1)
test_df["max"] = test_df[features].max(axis = 1)
test_df["mean"] = test_df[features].mean(axis = 1)
test_df["std"] = test_df[features].std(axis = 1)

### Getting Target variable and removing id

In [13]:
y = train_df["target"]
train_df = train_df.drop(["id"], axis=1)
train_df.drop(["target"], inplace = True, axis =1)

In [14]:
test_df = test_df.drop(["id"], axis = 1)

### Splitting the data

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(train_df, y, stratify = y, random_state = 123, test_size = 0.1)

### Scaling the data

In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Model trainin

In [34]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=123, C = 0.01, penalty = 'l2')
lr.fit(X_train, y_train)

In [37]:
from sklearn.metrics import roc_curve, auc
y_pred = lr.predict_proba(X_test)
y_pred = y_pred[:, 1] #is for log regression
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(auc(fpr, tpr))
# This is the score 0.8397180086841167

In [27]:
test_df = scaler.transform(test_df)

In [28]:
y_res = lr.predict_proba(test_df)

In [29]:
sub
y_res = y_res[:,-1] # for  logistic regression

In [30]:
sub["target"] = y_res
sub.to_csv("xg.csv", index=False)
sub.head(10)